##import the Wikidata integrator

In [ ]:
%%capture
!pip install wikidataintegrator
from wikidataintegrator import wdi_core, wdi_login

## Get all Wikidata properties

In [ ]:
query = """
SELECT DISTINCT ?property ?propertyLabel ?propertyDescription ?propType WHERE {
   ?property wikibase:directClaim ?p ;
             wikibase:propertyType ?propType ;
             schema:description ?propertyDescription ;
             rdfs:label ?propertyLabel .
   FILTER (lang(?propertyLabel) = 'en')
   FILTER (lang(?propertyDescription) = 'en')}
"""
propertiesSparql = wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe=True)
propertiesSparql
datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseLexeme': 'lexeme',
                'http://wikiba.se/ontology#WikibaseForm': 'form',
                'http://wikiba.se/ontology#WikibaseSense': 'sense',
                'http://wikiba.se/ontology#MusicalNotation': 'musical-notation',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}
propertiesSparql['datatype']= ""
for index, row in propertiesSparql.iterrows():
  row["datatype"] = datatype_map[row["propType"]] 
propertiesSparql

,property,propType,propertyLabel,propertyDescription,datatype
0,http://www.wikidata.org/entity/P364,http://wikiba.se/ontology#WikibaseItem,original language of film or TV show,language in which a film or a performance work...,wikibase-item
1,http://www.wikidata.org/entity/P10,http://wikiba.se/ontology#CommonsMedia,video,"relevant video. For images, use the property P...",commonsMedia
2,http://www.wikidata.org/entity/P610,http://wikiba.se/ontology#WikibaseItem,highest point,"point with highest elevation in a region, on a...",wikibase-item
3,http://www.wikidata.org/entity/P617,http://wikiba.se/ontology#String,yard number,identifier of a ship assigned by its builder (...,string
4,http://www.wikidata.org/entity/P618,http://wikiba.se/ontology#WikibaseItem,source of energy,describes the source of energy an animated obj...,wikibase-item
...,...,...,...,...,...
7838,http://www.wikidata.org/entity/P6921,http://wikiba.se/ontology#ExternalId,MangaSeek person ID,identifier for a mangaka on the MangaSeek website,external-id
7839,http://www.wikidata.org/entity/P6917,http://wikiba.se/ontology#ExternalId,Historical Archives of the European Union ID,identifier for a person or an institution that...,external-id
7840,http://www.wikidata.org/entity/P6918,http://wikiba.se/ontology#ExternalId,hitparade.ch artist ID,identifier for an artist on hitparade.ch music...,external-id
7841,http://www.wikidata.org/entity/P6915,http://wikiba.se/ontology#ExternalId,Gambay ID,identifier for a language or place in the Gamb...,external-id


In [ ]:
propertiesSparql.columns

Index(['property', 'propType', 'propertyLabel', 'propertyDescription'], dtype='object')

## Login to Wikibase

In [ ]:
from getpass import getpass
import pprint

wikibase = "https://demowb2.wiki.opencura.com/"
api = wikibase+"w/api.php"
sparql = wikibase+"query/sparql"
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

username:··········
Enter your password: ··········
Successfully logged in as Andrawaag


#### Function to create properties

In [ ]:
def createProperty(login=login, wdprop=None, label=label, description=description, property_datatype=property_datatype):
  if wdprop== None:
    s = []
  else:
    s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
  item = localEntityEngine(data=s)
  item.set_label(label)
  item.set_description(description)
  print(item.write(login, entity_type="property", property_datatype=property_datatype))

#create mapping to wikidata property

createProperty(login, label="property in wikidata", description="The same property in Wikidata", property_datatype="url")
  

#### Create Wikidata properties

In [ ]:
datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}

for row in propertiesSparql.itertuples():
  print(datatype_map[row.propType])
  print(row)
  createProperty(login=login, wdprop=row.property, label=row.propertyLabel, description=row.propertyDescription, property_datatype=datatype_map[row.propType])

external-id
Pandas(Index=0, property='http://www.wikidata.org/entity/P494', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='ICD-10', propertyDescription='identifier in the\xa0ICD catalogue codes for diseases - Version 10')


/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:227: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")
/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:175: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


P1
external-id
Pandas(Index=1, property='http://www.wikidata.org/entity/P493', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='ICD-9', propertyDescription='identifier in the ICD catalogue codes for diseases – Version 9')
P2
wikibase-item
Pandas(Index=2, property='http://www.wikidata.org/entity/P495', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='country of origin', propertyDescription='country of origin of this item (creative work, food, phrase, product, etc.)')
P3
wikibase-item
Pandas(Index=3, property='http://www.wikidata.org/entity/P6', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='head of government', propertyDescription='head of the executive power of this town, city, municipality, state, country, or other governmental body')
P4
wikibase-item
Pandas(Index=4, property='http://www.wikidata.org/entity/P361', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='part of', propertyDescription='object of which the subje

KeyError: ignored